[View in Colaboratory](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/index.ipynb)

# Combinación de bases de datos de artículos científicos
```
URL_base=https://colab.research.google.com/github/blob_file_in_repo
blob_file_in_repo → restrepo/medicion/blob/master/cienciometria/merge.ipynb

```
## UdeA
### Combinar bases de datos bibliográficas UDEA
* [Combibnar W0S+SCP+SCI](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/merge.ipynb) versión detallada
* [Combibnar W0S+SCP+SCI](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/WOS_SCI_SCP.ipynb) versión corta
##  [Open Access](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/OpenAccess.ipynb) DOAJ API 
##  [Open Access](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/XXX.ipynb)  

https://unpaywall.org/ dataset. See https://towardsdatascience.com/interactively-analyse-100gb-of-json-data-with-spark-e018f9436e76
```bash
# Select True Open Access articles
grep -E '"is_oa":\s*true' unpaywall_snapshot_2018-06-21T164548_with_versions.jsonl  >  oa.json
```

Further filters in `oa.json`
```bash
grep -E '"journal_is_oa":\s*true' oa.json > oagreen.json
grep -E '"journal_is_oa":\s*false' oa.json > oatmp.json
```
Further filters in `oatmp.json`
```bash
grep -F '"best_oa_location": {"url": "https://doi.org/' oatmp.json > oagreenarticle.json
grep -Fv '"best_oa_location": {"url": "https://doi.org/' oatmp.json > oayellow.json
```
Generate a cvs files with only  DOI column for each with the script `extractdois`:
```bash
#!/usr/bin/env bash
if [ ! "$2" ];then
    echo "USAGE: $0 filein fileout"
    exit
fi
echo DOI > "$2"
awk -F","  '{print $1}' "$1" |  grep '"doi":' | awk -F '"' '{print $(NF-1)}' >> "$2"
```
So:
```bash
extractdois  unpaywall_snapshot_2018-06-21T164548_with_versions.json dois.csv
extractdois  oa.json oadoi.csv
extractdois  oagreen.json oagreendois.csv
extractdois  oagreenarticle.json oagreenarticledois.csv
extractdois  oayellow,json oayellowdois.csv
```
Results here:
1. [oa.json](https://drive.google.com/file/d/1B6ndkSGNtHfPLm7CuCJMsoh_URHImK7v/view?usp=sharing)
1. [oayellow.json]()
1. [oagreen.json]()
1. [oagreenarticle.json]()
1. [dois.csv]()
1. [oadois.csv](https://drive.google.com/file/d/1uipr7JlNhLzZK5W8VymUNwVGGim-AqhC/view?usp=sharing)
1. [oayellowdois.csv](https://drive.google.com/file/d/1sBFXSOVXDZwqkI0SsUfNoU9ls138aHSw/view?usp=sharing)
1. [oagreendois.csv](https://drive.google.com/file/d/1RKB49-9xbTSuLxBZQ28JoQCzKNTr2tnU/view?usp=sharing)
1. [oagreenarticledois.csv](https://drive.google.com/file/d/1X5hr_q15UdoKKL_JJnLZR37r0UN2AS7y/view?usp=sharing)

* TODO: comprobar los APIS de http://blog.impactstory.org/introducting-oadoi/. Por ejemplo
Usar
 
 http://oadoi.org/10.1103/PhysRevD.98.055016

 En lugar de   http:://doi.org/10.1103/PhysRevD.98.055016
 
 To extract a line it is posible to ue the command
 ```bash
 n=5234678
 sed -n  ''$n','$n'p;'$n'q' oagreen.json
 ```

In [0]:
import pandas as pd

Open Acces colores
Yellow open Access
1.  Only artilcle in not oa journal

In [0]:
{"doi": "10.1103/physrevd.97.115032", "year": 2018, "genre": "journal-article", "is_oa": true, 
 "title": "Bound-state dark matter and Dirac neutrino masses", 
 "doi_url": "https://doi.org/10.1103/physrevd.97.115032", "updated": "2018-06-20T13:53:19.253782", 
 "publisher": "American Physical Society (APS)", "z_authors": [{"given": "M.", "family": "Reig", "sequence": "first"},
                                                               {"given": "D.", "family": "Restrepo", "sequence": "additional"}, 
                                                               {"given": "J. W. F.", "family": "Valle", "sequence": "additional"}, 
                                                               {"given": "O.", "family": "Zapata", "sequence": "additional"}], 
 "journal_name": "Physical Review D", 
 "oa_locations": [{"url": "https://doi.org/10.1103/physrevd.97.115032", "pmh_id": null,
                   "is_best": true, "license": "cc-by", "updated": "2018-06-20T13:53:19.252860", 
                   "version": "publishedVersion", "evidence": "open (via crossref license)", "host_type": "publisher", "url_for_pdf": null,
                   "url_for_landing_page": "https://doi.org/10.1103/physrevd.97.115032"}], 
 "data_standard": 1, "journal_is_oa": false, "journal_issns": "2470-0010,2470-0029", "published_date": "2018-06-19", 
 "best_oa_location": {"url": "https://doi.org/10.1103/physrevd.97.115032", "pmh_id": null, "is_best": true, "license": "cc-by", 
                      "updated": "2018-06-20T13:53:19.252860", "version": "publishedVersion", "evidence": "open (via crossref license)", 
                      "host_type": "publisher", "url_for_pdf": null, "url_for_landing_page": "https://doi.org/10.1103/physrevd.97.115032"}, 
 "journal_is_in_doaj": false}


2. No OA article with OA PDF version-

In [0]:
grayoa=pd.DataFrame()
grayoa.append( {"doi": "10.1103/physrevd.86.075011", "year": 2012, "genre": "journal-article", "is_oa": True, 
              "title": "Connection of gamma rays, dark matter, and Higgs boson searches at the LHC",
              "doi_url": "https://doi.org/10.1103/physrevd.86.075011", "updated": "2018-06-18T03:46:10.784962",
              "publisher": "American Physical Society (APS)", "z_authors": [{"given": "J. D.", "family": "Ruiz-Alvarez"}, 
                                                                            {"given": "C. A.", "family": "de S. Pires"}, 
                                                                            {"given": "Farinaldo S.", "family": "Queiroz"}, 
                                                                            {"given": "D.", "family": "Restrepo"}, 
                                                                            {"given": "P. S.", "family": "Rodrigues da Silva"}], 
              "journal_name": "Physical Review D", 
              "oa_locations": [{"url": "http://arxiv.org/pdf/1206.5779", "pmh_id": "oai:arXiv.org:1206.5779",
                                                                     "is_best": True, "license": 'null', "updated": "2018-06-18T03:46:10.783600", 
                                                                     "version": "submittedVersion", "evidence": "oa repository (via OAI-PMH doi match)", 
                                                                     "host_type": "repository", "url_for_pdf": "http://arxiv.org/pdf/1206.5779",
                                                                     "url_for_landing_page": "http://arxiv.org/abs/1206.5779"}], 
              "data_standard": 2, "journal_is_oa": False, "journal_issns": "1550-7998,1550-2368", "published_date": "2012-10-08", 
              "best_oa_location": {"url": "http://arxiv.org/pdf/1206.5779", "pmh_id": "oai:arXiv.org:1206.5779", "is_best": True, "license": 'null',
                                   "updated": "2018-06-18T03:46:10.783600", "version": "submittedVersion", 
                                   "evidence": "oa repository (via OAI-PMH doi match)", "host_type": "repository", 
                                   "url_for_pdf": "http://arxiv.org/pdf/1206.5779", "url_for_landing_page": "http://arxiv.org/abs/1206.5779"}, 
              "journal_is_in_doaj": False},ignore_index=True)


,best_oa_location,data_standard,doi,doi_url,genre,is_oa,journal_is_in_doaj,journal_is_oa,journal_issns,journal_name,oa_locations,published_date,publisher,title,updated,year,z_authors
0,"{u'updated': u'2018-06-18T03:46:10.783600', u'...",2.0,10.1103/physrevd.86.075011,https://doi.org/10.1103/physrevd.86.075011,journal-article,1.0,0.0,0.0,"1550-7998,1550-2368",Physical Review D,"[{u'updated': u'2018-06-18T03:46:10.783600', u...",2012-10-08,American Physical Society (APS),"Connection of gamma rays, dark matter, and Hig...",2018-06-18T03:46:10.784962,2012.0,"[{u'given': u'J. D.', u'family': u'Ruiz-Alvare..."


1.  OA artilcle in OA  journal

In [0]:
{"doi": "10.1371/journal.pone.0144703.g008", "year": null, "genre": "component", "is_oa": true, 
 "title": null, "doi_url": "https://doi.org/10.1371/journal.pone.0144703.g008", "updated": "2018-06-15T18:17:52.197555", 
 "publisher": "Public Library of Science (PLoS)", "z_authors": null, 
 "journal_name": null, "oa_locations": [{"url": "https://doi.org/10.1371/journal.pone.0144703.g008", "pmh_id": null, "is_best": true, "license": null, "updated": "2018-06-21T00:53:21.144034", "version": "publishedVersion", "evidence": "oa journal (via publisher name)", "host_type": "publisher", "url_for_pdf": null, "url_for_landing_page": "https://doi.org/10.1371/journal.pone.0144703.g008"}], "data_standard": 2, "journal_is_oa": true, "journal_issns": null, "published_date": null, "best_oa_location": {"url": "https://doi.org/10.1371/journal.pone.0144703.g008", "pmh_id": null, "is_best": true, "license": null, "updated": "2018-06-21T00:53:21.144034", "version": "publishedVersion", "evidence": "oa journal (via publisher name)", "host_type": "publisher", "url_for_pdf": null, "url_for_landing_page": "https://doi.org/10.1371/journal.pone.0144703.g008"}, "journal_is_in_doaj": false}

In [0]:
from bs4 import BeautifulSoup

In [0]:
import requests
issn='2045-2322'#'0960-9822'#'2211-3797'
url='https://doaj.org/toc/{}'.format(issn)
r=requests.get(url)

In [0]:
soup=BeautifulSoup(r.text)

In [0]:
try:
    APC=soup.find_all('div',{"class":"box slim"})[0].contents[3].contents[2].strip()
except:
    APC=None

In [0]:
APC

u'1165GBP'

In [0]:
!pip install currencyconverter > /dev/null

See https://pypi.org/project/CurrencyConverter/

In [0]:
from currency_converter import CurrencyConverter
c = CurrencyConverter()

In [0]:
CurrencyConverter()

In [0]:
i=re.search(r"([0-9\.\,]+)([a-zA-Z]+)",APC)
items=i.groups()

In [0]:
items


(u'1165', u'GBP')

In [0]:
int(  c.convert(float(items[0]),items[1],'USD'))

1510

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("drive/My Drive/Colab Notebooks/Data")

In [0]:
!grep -E '"journal_issns":\s+"' oagreen.json | tail

{"doi": "10.22329/wyaj.v31i2.4418", "year": 2013, "genre": "journal-article", "is_oa": true, "title": "SHIFTING JUDICIAL CONCEPTIONS OF 'RECONCILIATION': GEOGRAPHIC COMMITMENTS UNDERPINNING ABORIGINAL RIGHTS DECISIONS", "doi_url": "https://doi.org/10.22329/wyaj.v31i2.4418", "updated": "2018-06-17T15:36:19.230472", "publisher": "University of Windsor Leddy Library", "z_authors": [{"given": "Michael", "family": "McCrossan"}], "journal_name": "Windsor Yearbook of Access to Justice", "oa_locations": [{"url": "https://doi.org/10.22329/wyaj.v31i2.4418", "pmh_id": null, "is_best": true, "license": "cc-by", "updated": "2018-06-17T15:36:19.230009", "version": "publishedVersion", "evidence": "oa journal (via doaj)", "host_type": "publisher", "url_for_pdf": null, "url_for_landing_page": "https://doi.org/10.22329/wyaj.v31i2.4418"}], "data_standard": 1, "journal_is_oa": true, "journal_issns": "0710-0841", "published_date": "2013-10-01", "best_oa_location": {"url": "https://doi.org/10.22329/wyaj.v31

```bash
awk -F'", "published_date"' '{print $1}' kk.json | awk -F'"journal_issns": "' '{print $2}' | perl -p -e 's/,/\n/' | sort | uniq
```

In [0]:
!echo "DOI,ISSNs" > oagreendoissn.csv

In [0]:
! awk -F'"published_date"' '{print $1}' oagreen.json | awk -F":" '{print $2 $NF}' | sed -r 's/\s+"year"\s+//' | perl -p -e 's/,\s*\n/\n/' >> oagreendoissn.csv

In [0]:
ls -lh oagreendoissn.csv

-rw------- 1 root root 224M Oct 19 22:08 oagreendoissn.csv


In [0]:
!pip install openpyxl > /dev/null

In [0]:
import pandas as pd

In [0]:
ds=pd.read_csv('oagreendoissn.csv')

In [0]:
ds['DOI']=ds['DOI'].str.replace('"','')

In [0]:
ds=ds.fillna('')

In [0]:
kk=ds[ ds['ISSNs']!='' ][:3]

In [0]:
import numpy as np

In [0]:
issns=np.unique(  ds.ISSNs.str.split(',').values.sum() )
issns.shape

In [0]:
 lenmax=ds['ISSNs'].str.split(",").apply(len).max()

In [0]:
for i in range(lenmax):
    ds['ISSN{}'.format(i)]=ds['ISSNs'].str.split(",").str[i]

In [0]:
ds=ds.fillna('')

In [0]:
ds.to_excel('oagreendoissn.xlsx',index=False)

In [1]:
ls

drive/  sample_data/
